GOSSのシート

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib

In [2]:
###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')
buildings = pd.read_csv('buildings.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_buildings = pd.read_csv('test_buildings.csv')

GOSSで契約期間２＆間取り（S以外）

In [4]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K"]]], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K"]]], axis=1)

category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'goss',
    'metrics':'rmse',
    'reg_lambda': 1.2667345328336822e-06,
    'max_bin': 405,
    'num_leaves': 118
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=1000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future rel

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2120
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35106.1	valid_1's rmse: 40847.8
[20]	training's rmse: 22594.6	valid_1's rmse: 31412.3
[30]	training's rmse: 17865.3	valid_1's rmse: 28008.8
[40]	training's rmse: 15811.4	valid_1's rmse: 26544
[50]	training's rmse: 14540	valid_1's rmse: 25658.7
[60]	training's rmse: 13552.1	valid_1's rmse: 25125.6
[70]	training's rmse: 12747.8	valid_1's rmse: 24738.1
[80]	training's rmse: 12054.9	valid_1's rmse: 24480.1
[90]	training's rmse: 11508.5	valid_1's rmse: 24211.2
[100]	training

In [5]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('GOSS_cs2_nLDK_result.csv',index=False)

In [6]:
LGBM_cs2_nLD_bld_results = pd.read_csv('LGBM_cs2_nLD_bld_result.csv')
DART_cs_nLDKS_bld_results = pd.read_csv('DART_cs_nLDKS_bld_result.csv')
GOSS_cs2_nLDK_results = pd.read_csv('GOSS_cs2_nLDK_result.csv')
X = pd.concat([LGBM_cs2_nLD_bld_results['0'],GOSS_cs2_nLDK_results['0']],axis=1)
X.corr()

,0,0
0,1.000000,0.985823
0,0.985823,1.000000


GOSSで契約期間1＆フル間取り＆建物構造

In [7]:
X_train = pd.concat([house_age, area_size, contract_span, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]],buildings], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_addresses, test_room_arrange[["部屋数","L","D","K","S"]],test_buildings], axis=1)

category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'K','S','建物構造']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'goss',
    'metrics':'rmse',
    'reg_lambda': 1.2667345328336822e-06,
    'max_bin': 405,
    'num_leaves': 118
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=1000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '建物構造', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001878 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2126
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 15
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35426.3	valid_1's rmse: 41103.3
[20]	training's rmse: 23000.4	valid_1's rmse: 31798.8
[30]	training's rmse: 17820.2	valid_1's rmse: 28046.6
[40]	training's rmse: 15776.3	valid_1's rmse: 26583.6
[50]	training's rmse: 14394.5	valid_1's rmse: 25723.2
[60]	training's rmse: 13467.2	valid_1's rmse: 25201.1
[70]	training's rmse: 12642.5	valid_1's rmse: 24857
[80]	training's rmse: 12038.4	valid_1's rmse: 24556.1
[90]	training's rmse: 11451.2	valid_1's rmse: 24290.2
[100]	traini

In [8]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('GOSS_cs_nLDKS_bld_result.csv',index=False)

In [9]:
LGBM_cs2_nLD_bld_results = pd.read_csv('LGBM_cs2_nLD_bld_result.csv')
DART_cs_nLDKS_bld_results = pd.read_csv('DART_cs_nLDKS_bld_result.csv')
GOSS_cs_nLDKS_results_bld = pd.read_csv('GOSS_cs_nLDKS_bld_result.csv')
X = pd.concat([LGBM_cs2_nLD_bld_results['0'],GOSS_cs_nLDKS_results_bld['0']],axis=1)
X.corr()

,0,0
0,1.000000,0.985526
0,0.985526,1.000000


lightGBMの特徴量としてGOSSの予測値を使うための処理

In [10]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, addresses, room_arrange[["部屋数","L","D","K","S"]],buildings], axis=1)
y_pred = model.predict(X_train, num_iteration=model.best_iteration)

In [13]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('en_GOSS_cs_nLDKS_bld_result.csv',index=False)

In [14]:
y_pred

,0
0,88384.821604
1,70266.789393
2,108856.517348
3,153591.449643
4,86642.142535
...,...
31465,77901.171395
31466,57818.216723
31467,112098.839195
31468,100056.789927
